In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pathlib
os.getcwd()
df = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train.csv')
# pathlib.Path().resolve()
# pathlib.Path().parent.resolve()
# pathlib.Path().absolute()
print (len(df))
df.head()


In [ ]:
from catboost import CatBoostClassifier   #, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X = df.drop(columns=["is_attributed", "click_time", "attributed_time"])
y = df["is_attributed"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

cb = CatBoostClassifier(n_estimators=200,
                       loss_function="Logloss", eval_metric="AUC",
                       depth=3, task_type='CPU',
                       random_state=42,
                       verbose=200, auto_class_weights="Balanced")

cb.fit(X_train, y_train, cat_features = ["ip", "app", "device", "os"], plot=True)

In [ ]:
y_pred = cb.predict(X_test, prediction_type="Probability")

roc_auc_score(y_pred[:, 1], y_test)

In [ ]:
del (df, X_train, X_test, y_train, y_test, X, y)

In [ ]:
sample = pd.read_csv("/input/talkingdata-adtracking-fraud-detection/sample_submission.csv")

sample.head()

In [ ]:
test = pd.read_csv("/kaggle/input/ad-tracking-fraud-detection/test.csv")

preds_to_publish = cb.predict(test, prediction_type="Probability")